In [13]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from statistics import NormalDist

from classiq import *
from classiq.applications.iqae.iqae import IQAE

**Classical Monte Carlo**

Our return distribution is modeled by a normal distribution with mean of 0.15 (15%) and standard deviation of 0.20 (20%). 

In [14]:
def build_normal_return_grid_probs(mu=0.15, sigma=0.20, n_qubits=7, k_sigmas=4.0):
    """
    Build a discretized 1Y return distribution R ~ N(mu, sigma^2) on an N=2^n grid.

    We create N equal-width bins spanning [mu - k_sigmas*sigma, mu + k_sigmas*sigma].
    Probability mass for each bin is computed via Normal CDF differences, then renormalized
    to account for truncation.

    Returns:
      grid_centers: shape (N,) return value representative for each bin (midpoint)
      probs:        shape (N,) probability mass per bin (sums to 1)
      edges:        shape (N+1,) bin edges
      meta:         dict with useful parameters
    """
    N = 2 ** n_qubits
    r_min = mu - k_sigmas * sigma
    r_max = mu + k_sigmas * sigma

    edges = np.linspace(r_min, r_max, N + 1)
    grid_centers = 0.5 * (edges[:-1] + edges[1:])

    # Normal CDF
    nd = NormalDist(mu=mu, sigma=sigma)
    cdf = np.array([nd.cdf(x) for x in edges])

    probs = np.diff(cdf)               # bin masses
    probs = np.maximum(probs, 0.0)     # numerical safety, needed if a numerical error occur
    probs = probs / probs.sum()        # renormalize after truncation

    meta = {
        "mu": mu,
        "sigma": sigma,
        "n_qubits": n_qubits,
        "N": N,
        "k_sigmas": k_sigmas,
        "r_min": r_min,
        "r_max": r_max,
        "bin_width": (r_max - r_min) / N,
        "trunc_mass": float(cdf[-1] - cdf[0]),  # mass inside [r_min, r_max] before renorm
    }
    return grid_centers, probs, edges, meta


# Example usage
num_qubits = 6
grid, probs, edges, meta = build_normal_return_grid_probs(mu=0.15, sigma=0.20, n_qubits=num_qubits, k_sigmas=4.0)

print("Meta:", meta)
print("Sum probs:", probs.sum())
print("First 5 grid points:", grid[:5])
print("First 5 probs:", probs[:5])


Meta: {'mu': 0.15, 'sigma': 0.2, 'n_qubits': 6, 'N': 64, 'k_sigmas': 4.0, 'r_min': -0.65, 'r_max': 0.9500000000000001, 'bin_width': 0.025, 'trunc_mass': 0.9999366575163338}
Sum probs: 1.0
First 5 grid points: [-0.6375 -0.6125 -0.5875 -0.5625 -0.5375]
First 5 probs: [2.16424788e-05 3.51071592e-05 5.60669921e-05 8.81539370e-05
 1.36458019e-04]


To estimate the VaR using the **quantum amplitude algorithm** and its variants, we first discretize the loss distribution into $N=2^n$ intervals (where $n$ is the number of qubits), and assign a state $\ket{i}$ to , with .

In [15]:
ALPHA = 0.05          # 95% confidence level
TOLERANCE = 0.001     # Convergence tolerance
grid_points = grid    # Rename to match function expectations
# Calculate the true VaR for comparison (5th percentile of normal distribution)
from scipy.stats import norm
mu, sigma = 0.15, 0.20
VAR = norm.ppf(ALPHA, loc=mu, scale=sigma)  # Should be around -0.179
# Make qubit counts consistent
num_qubits = 6  # Match the distribution grid size

In [54]:
# Given a list of probabilities This function calculates the alpha classically given the index and the list of probabilities
def calc_alpha(index: int, probs: list[float]):
    sum_probs = sum([probs[i] for i in range(index)])
    return sum_probs

def update_index(index: int, required_alpha: float, alpha_v: float, search_size: int):
    if alpha_v < required_alpha:
        return index + search_size
    return index - search_size

def print_status(v, alpha_v, search_size, index):
    print(f"v: {v}, alpha_v: {alpha_v}")
    print(f"{search_size=}")
    print(f"{index=}")
    print("------------------------")

def print_results(grid_points, index, probs):
    print(f"Value at risk at {ALPHA*100}%: {grid_points[index]}")
    global VAR
    print(f"Real VaR", VAR)
    return index

def value_at_risk(required_alpha, index, calc_alpha_func=calc_alpha):
    v = probs[index]
    alpha_v = calc_alpha_func(index, probs)
    search_size = index // 2
    print_status(v, alpha_v, search_size, index)

    # Tolerance represents the accuracy of the alpha we aim to get
    while (not np.isclose(alpha_v, required_alpha, atol=TOLERANCE)) and search_size > 0:
        index = update_index(index, required_alpha, alpha_v, search_size)
        # Binary search, divided by 2 - as we know the function is always growing in that part of the graph.
        search_size = search_size // 2
        v = grid_points[index]
        alpha_v = calc_alpha_func(index, probs)
        print_status(v, alpha_v, search_size, index)

    print_results(grid_points, index, probs)

Over a time horizon of a year and a confidence interval of (0.95) 95%, we use a Monte Carlo simulation to estimate the value at risk (VaR) of our return distribution.

**Iterative Quantum Amplitude Estimation (QAE)**

The goal of QAE is to estimate the probability (or amplitude) of a certain outcome from a quantum circuit. In our case, we will use this to estimate our value at risk.

In [34]:
@qfunc(synthesize_separately=True)
def state_preparation(asset: QArray[QBit], ind: QBit):
    load_distribution(asset=asset)
    payoff(asset=asset, ind=ind)

@qfunc
def load_distribution(asset: QNum):
    inplace_prepare_state(probs, bound=0, target=asset)

@qperm
def payoff(asset: Const[QNum], ind: QBit):
    ind ^= asset < GLOBAL_INDEX

In [18]:
written_qmod = False


# Using IQAE
def calc_alpha_quantum(index: int, probs: list[float]):

    # Global variable
    global GLOBAL_INDEX
    GLOBAL_INDEX = index

    # Creation of the model, given the constratins and the circuit preferences
    iqae = IQAE(
        state_prep_op=state_preparation,
        problem_vars_size=num_qubits,
        constraints=Constraints(max_width=28),
        preferences=Preferences(machine_precision=num_qubits),
    )

    qprog = iqae.get_qprog()

    global written_qmod
    qmod = iqae.get_model()
    if not written_qmod:
        written_qmod = True
        show(qprog)

    iqae_res = iqae.run(epsilon=0.05, alpha=0.01)

    # Result of the iterative QAE
    # iqae_res = res[0].value
    measured_payoff = iqae_res.estimation
    confidence_interval = np.array(
        [interval for interval in iqae_res.confidence_interval]
    )
    print("Measured Payoff:", measured_payoff)
    print("Confidence Interval:", confidence_interval)
    return measured_payoff

In [ ]:
num_qubits = 6
GLOBAL_INDEX = 0  # Initialize before use in payoff

def get_initial_index():
    return int(2**num_qubits) // 4

index = get_initial_index()

# Use quantum amplitude estimation instead of classical
var_index = value_at_risk(0.05, index, calc_alpha_func=calc_alpha_quantum)

Quantum program link: https://platform.classiq.io/circuit/393ts82f70zCscrUZ1NPtihfWiy


https://platform.classiq.io/circuit/393ts82f70zCscrUZ1NPtihfWiy?login=True&version=16

Measured Payoff: 0.02219516505064606
Confidence Interval: [0.01986016 0.02453017]
v: 0.003592222281243454, alpha_v: 0.02219516505064606
search_size=16
index=32
------------------------
Measured Payoff: 0.1578818313701744
Confidence Interval: [0.15075391 0.16500976]
v: -0.043749999999999956, alpha_v: 0.1578818313701744
search_size=8
index=48
------------------------
Measured Payoff: 0.06670501815189181
Confidence Interval: [0.06451169 0.06889835]
v: -0.14375, alpha_v: 0.06670501815189181
search_size=4
index=40
------------------------
Measured Payoff: 0.04234983133017358
Confidence Interval: [0.03872052 0.04597914]
v: -0.19375, alpha_v: 0.04234983133017358
search_size=2
index=36
------------------------
Measured Payoff: 0.05307191689617329
Confidence Interval: [0.04821749 0.05792635]
v: -0.16874999999999998, alpha_v: 0.05307191689617329
search_size=1
index=38
------------------------
Measured Payoff: 0.04684916153240862
Confidence Interval: [0.0427742  0.05092412]
v: -0.18125, alpha_v: 

QAE



In [44]:
num_qubits = 6
num_phase_qubits = 3

In [45]:
# S0 flip operator
@qfunc
def prepare_minus(q: QBit):
    X(q)
    H(q)

@qfunc
def S0(x: QNum, ind: QBit):
    within_apply(lambda: prepare_minus(ind), lambda: inplace_xor(x == 0, ind))

@qfunc
def qpe(unitary: QCallable[CInt], phase: QArray[QBit]):
    apply_to_all(H, phase)

    repeat(
        count=phase.len,
        iteration=lambda i: control(
            ctrl=phase[i],
            stmt_block=lambda: unitary(2**i),
        ),
    )

    invert(lambda: qft(phase))


@qfunc
def grover_op(
    power: CInt,
    asset: QArray[QBit],
    ind: QBit,
):
    repeat(
        count=power,
        iteration=lambda _: (
            Z(ind),  # S_chi
            invert(lambda: state_preparation(asset, ind)),  # A†
            S0(asset, ind),  # S0
            state_preparation(asset, ind) # A
        )
    )


In [ ]:
@qfunc
def main(phase: Output[QNum[num_phase_qubits, UNSIGNED, num_phase_qubits]]):

    asset = QArray()
    allocate(num_qubits, asset)

    ind = QBit()
    allocate(ind)

    # THIS is the initial state
    state_preparation(asset, ind)

    allocate(phase)

    # QPE now runs on whatever state exists
    qpe(
        unitary=lambda p: grover_op(p, asset, ind),
        phase=phase
    )

    drop(asset)
    drop(ind)

In [ ]:
def calc_alpha_qae(index: int) -> float:
    """
    Rebuild the QAE circuit for the given index and return measured probability α_v.
    """
    global GLOBAL_INDEX
    GLOBAL_INDEX = index

    @qfunc
    def main(phase: Output[QNum[num_phase_qubits, UNSIGNED, num_phase_qubits]]):
        asset = QArray()
        allocate(num_qubits, asset)

        ind = QBit()
        allocate(ind)

        allocate(phase)

        # Use the existing initial state
        state_preparation(asset, ind)

        # Run QPE on Grover operator
        qpe(
            unitary=lambda p: grover_op(p, asset, ind),
            phase=phase
        )

        drop(asset)
        drop(ind)

    # Build and synthesize circuit
    model = create_model(main)
    model = set_constraints(model, Constraints(max_width=28))
    qprog = synthesize(model)
    result = execute(qprog).result_value()

    # Extract measured phase → alpha_v
    phases_counts = {sampled_state.state["phase"]: sampled_state.shots
                     for sampled_state in result.parsed_counts}
    best_phase = max(phases_counts, key=phases_counts.get)
    phi = best_phase / (2**num_phase_qubits)
    alpha_v = np.sin(np.pi * phi) ** 2
    return alpha_v

In [ ]:
def value_at_risk_QAE(required_alpha: float, initial_index: int, tol: float = 0.005, max_iter: int = 20):
    index = initial_index
    search_size = index // 2
    iteration = 0

    while iteration < max_iter and search_size > 0:
        v = grid_points[index]
        alpha_v = calc_alpha_qae(index)
        print(f"v: {v}, alpha_v: {alpha_v}, search_size={search_size}, index={index}")
        
        # Binary search update
        if alpha_v < required_alpha:
            index += search_size
        else:
            index -= search_size
        search_size //= 2
        iteration += 1

    # Final result
    v_final = grid_points[index]
    alpha_final = calc_alpha_qae(index)
    print(f"Value at risk at {required_alpha*100}%: {v_final}")
    print(f"Measured alpha_v: {alpha_final}")
    print()
    return v_final, alpha_final


In [51]:
num_qubits = 6
num_phase_qubits = 3

GLOBAL_INDEX = 0  # Initialize before use in payoff

def get_initial_index():
    return int(2**num_qubits) // 4

v_final, alpha_final = value_at_risk_QAE(required_alpha=0.05, initial_index=get_initial_index())

v: -0.2375, alpha_v: 0.03806023374435662, search_size=8, index=16
v: -0.03749999999999998, alpha_v: 0.059039367825822475, search_size=4, index=24
v: -0.1375, alpha_v: 0.059039367825822475, search_size=2, index=20
v: -0.1875, alpha_v: 0.03806023374435662, search_size=1, index=18
Value at risk at 5.0%: -0.1625
Measured alpha_v: 0.021529832133895567


In [53]:
print_results(grid_points, grid_points.tolist().index(v_final), probs)

Value at risk at 5.0%: -0.1625)
Real VaR -0.1789707253902946


19